In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix, vstack
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import f1_score
from implicit.als import AlternatingLeastSquares

In [2]:
df = pd.read_csv(r'd:\train\train.csv' , parse_dates = ['order_completed_at'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3123064 entries, 0 to 3123063
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   user_id             int64         
 1   order_completed_at  datetime64[ns]
 2   cart                int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 71.5 MB


,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157


In [3]:
%%time
# создаю таблицу без выбросов
df_ = df.groupby(['user_id', 'cart']).count().reset_index()
for i in df_['cart'].unique():
    df_[df_['cart']== i]['order_completed_at'].describe()
    q =  df_['order_completed_at'].describe()
    q1 = q['25%']
    q3 = q['75%']
    mq = q3 - q1
    filter_up = (df_['cart']== i) & (df_['order_completed_at'] > (q3 + mq*1.5))
    filter_down = (df_['cart']== i) &  (0 < df_['order_completed_at']) & (df_['order_completed_at'] < (q1 - mq*1.5))

    df_.loc[filter_up, 'user_id'] = float('Nan')
    df_.loc[filter_down, 'user_id'] = float('Nan')
df_.dropna(axis=0,inplace=True)

Wall time: 46.9 s


In [4]:
%%time
# создать матрицу со всеми категориями, а потом удалить что признано выбросами
q = df.groupby(['user_id', 'cart']).count().reset_index()
q = q.merge(df_, how = 'left', on = ['user_id','cart'])
q.columns = ['user_id', 'cart', 'kol', 'kol1']
q['kol'] = q.apply(lambda x: x.kol if x.kol == x.kol1 else 0, axis = 1)
q = q.pivot(index = 'user_id', columns = 'cart', values = 'kol')
q.fillna(0, inplace = True)
q.info()
q.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Columns: 881 entries, 0 to 880
dtypes: float64(881)
memory usage: 134.6 MB
Wall time: 28 s


cart,0,1,2,3,4,5,6,7,8,9,...,871,872,873,874,875,876,877,878,879,880
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test = pd.read_csv(r'd:\sample_submission.csv')
new_df = test['id'].str.split(';', expand = True)

new_df.columns=['user_id','cart']
new_df['user_id'] = new_df['user_id'].astype(int)
new_df['cart'] = new_df['cart'].astype(int)
new_df['target'] = test['target'] 
new_df.info()
new_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790449 entries, 0 to 790448
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  790449 non-null  int32
 1   cart     790449 non-null  int32
 2   target   790449 non-null  int64
dtypes: int32(2), int64(1)
memory usage: 12.1 MB


,user_id,cart,target
0,0,133,0
1,0,5,1
2,0,10,0
3,0,396,1
4,0,14,0


In [12]:
%%time
data = vstack([coo_matrix(q.to_numpy())]).tocsr()
data.shapecsr_data = csr_matrix(q.values)
model = AlternatingLeastSquares(factors = 256, regularization  = 0.5, iterations  = 10, calculate_training_loss  = True)
model.fit(data.T)

kol = list(q.columns) 
# закэшировать результаты рекоммендаций по пользователю
tab = new_df.groupby(['user_id'])['cart'].unique()
cash = {} 

for id in tab.index:
    w = set(kol.copy())
    w.difference_update(set(tab.loc[id]))
    cash_cart = {}
    for i in model.recommend(id, data, filter_already_liked_items = False, N=data.shape[1], filter_items= w):
        cash_cart[i[0]] = i[1]
    cash[id] = cash_cart
    

  0%|          | 0/10 [00:00<?, ?it/s]

Wall time: 20.6 s


In [13]:
%%time
def predict(id, cart):
    prob = cash.get(id).get(cart)
    return prob > 0.5

new_df['predict'] = new_df.apply(lambda x:  predict(x.user_id, x.cart), axis = 1)
f1_score(new_df['target'] , new_df['predict'])

Wall time: 18.2 s


0.633606718670523

In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(new_df['target'],  new_df['predict'])

array([[ 54403, 340191],
       [ 54545, 341310]], dtype=int64)

In [15]:
# константная модель
new_df['predict'] = 1
f1_score(new_df['target'] , new_df['predict'])

0.6673753102071645

In [16]:
confusion_matrix(new_df['target'],  new_df['predict'])

array([[     0, 394594],
       [     0, 395855]], dtype=int64)